In [0]:
# This notebook was made on google colab.
# Make sure to change runtime to GPU. 
# As the memory limit is 12GB, running all cells together on colab will crash runtime. 
# So the notebook should be partially run and the session needs to be reset
# The cells corresponding to taking the input need to be run everytime the session is started

# test and training images were split into two zips
# You need to make a copy of the files in your drive and then replace the link with corresponding links of your id

# The images were scaled to 48*64 size
# The model was tested with 1, 2 and 3 CNN layers and 2 layers were choosen
# Then different kernel size were tested and 72 for first and 88 for second layer was choosen
# Then different dense layer size were tested and 2048 was choosen
# Dropout was tested but it didn't improve result
# Converting second CNN layer (88C5) to two CNN layers (88C3-88C3) didn't improved result
# 2048 dense layer was split into two, 1024 and 128 gave best result 

#The initial cells corrsespond to these tests

#The model was run for 2000 epochs in two phases.
#After training was completed, model with lowest val_loss was selected for prediction
#The final prediction values were rounded off to nearest integer
#As weights are randomly assigned, it should be run for 2-3 time for best results
#TO TEST ONLY FINAL MODEL, SKIP THE CELLS MARKED "#MODEL TESTING"

In [1]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
import os, sys
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Link :https://drive.google.com/open?id=10yfJC1Ilkyj0GiOvHDCw9c-HMteOcvlJ
file_id = '10yfJC1Ilkyj0GiOvHDCw9c-HMteOcvlJ' #link to train images zip
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_images.zip')

#Link :https://drive.google.com/open?id=
train_id = '14auPVokQCaJ5VbcA3tPap2mfKaGFlnLd' #link to train.csv
myfile = drive.CreateFile({'id': train_id})
myfile.GetContentFile('file.csv')
train = pd.read_csv('file.csv')

    100% |████████████████████████████████| 993kB 20.3MB/s 


In [0]:
import zipfile, os
import pandas as pd
import numpy as np
import os, sys
train = pd.read_csv('file.csv')
zip_ref = zipfile.ZipFile('train_images.zip', 'r')
zip_ref.extractall('MODEL/')
zip_ref.close()

In [0]:
y = train.iloc[:, 1:5]

In [4]:
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from time import time
from time import sleep
folder = "/content/MODEL/train_images/"
train_files = train.iloc[:, 0:1]

print("Working with {0} images".format(len(train_files)))
train_files.head()

Working with 14000 images


,image_name
0,JPEG_20160706_121146_1000145715002.png
1,JPEG_20161119_174038_1000690577600.png
2,147444927651111470309333776-Roadster-Men-Casua...
3,147772332675720161028_161611.png
4,1473315333651DeeplearnS11638.png


In [5]:
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
image_width = 64
image_height = 48
channels = 3
dataset = np.ndarray(shape=(len(train_files), image_height, image_width, channels),
                     dtype=np.float32)
i = 0
for file in train_files.loc[:, "image_name"]:
    img = load_img(folder + "/" + file)  # this is a PIL image
    # Convert to Numpy Array
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((48, 64, 3))
    x = (x - 128.0) / 128.0
    dataset[i] = x;
    i+=1

    

Using TensorFlow backend.


In [0]:
!rm -R /content/MODEL/

In [0]:
!rm /content/train_images.zip

In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=0.01, random_state=37) #test_size was 0.1 for testing various models. 0.01 for final model
dataset = np.ndarray(shape=(1, image_height, image_width, channels),dtype=np.float32)

In [0]:
#MODEL TESTING
nets = 3
model = [0] *nets

for j in range(3):
    model[j] = Sequential()
    model[j].add(Conv2D(60,kernel_size=5,padding='same',activation='relu',input_shape=(image_height, image_width, channels)))
    model[j].add(MaxPool2D())
    if j>0:
        model[j].add(Conv2D(60,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D())
    if j>1:
        model[j].add(Conv2D(60,kernel_size=5,padding='same',activation='relu'))
        model[j].add(MaxPool2D(padding='same'))
    model[j].add(Flatten())
    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(4))
    model[j].compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])
    model[j].fit(X_train, y_train, batch_size=64, epochs=20, verbose = 1, validation_data = (X_test, y_test))

Train on 12600 samples, validate on 1400 samples
Epoch 1/20
12600/12600 [==============================] - 6s 497us/sample - loss: 9290.9828 - acc: 0.9005 - val_loss: 2802.1430 - val_acc: 0.9050
Epoch 2/20
12600/12600 [==============================] - 6s 464us/sample - loss: 2241.1649 - acc: 0.9110 - val_loss: 2041.9059 - val_acc: 0.9150
Epoch 3/20
12600/12600 [==============================] - 6s 462us/sample - loss: 1762.3325 - acc: 0.9179 - val_loss: 1715.5944 - val_acc: 0.9186
Epoch 4/20
12600/12600 [==============================] - 6s 464us/sample - loss: 1513.8817 - acc: 0.9250 - val_loss: 1616.7276 - val_acc: 0.9271
Epoch 5/20
12600/12600 [==============================] - 6s 465us/sample - loss: 1355.2508 - acc: 0.9301 - val_loss: 1360.0364 - val_acc: 0.9321
Epoch 6/20
12600/12600 [==============================] - 6s 463us/sample - loss: 1276.2565 - acc: 0.9325 - val_loss: 1303.5008 - val_acc: 0.9329
Epoch 7/20
12600/12600 [==============================] - 6s 464us/sample -

In [0]:
#MODEL TESTING
nets = 6
model = [0] *nets
for j in range(6):
    model[j] = Sequential()
    model[j].add(Conv2D(j*8 + 32,kernel_size=5,padding='same',activation='relu',input_shape=(image_height, image_width, channels)))
    model[j].add(Conv2D(j*8 + 48,kernel_size=5,padding='same', activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    model[j].add(Dense(256, activation='relu'))
    model[j].add(Dense(4))
    model[j].compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])
    model[j].fit(X_train, y_train, batch_size=64, epochs=20, verbose = 1, validation_data = (X_test, y_test))

Train on 12600 samples, validate on 1400 samples
Epoch 1/20
12600/12600 [==============================] - 9s 721us/sample - loss: 8266.1040 - acc: 0.9025 - val_loss: 2482.8553 - val_acc: 0.9000
Epoch 2/20
12600/12600 [==============================] - 8s 660us/sample - loss: 2061.5194 - acc: 0.9129 - val_loss: 1933.0520 - val_acc: 0.9064
Epoch 3/20
12600/12600 [==============================] - 8s 657us/sample - loss: 1761.6067 - acc: 0.9184 - val_loss: 1679.1044 - val_acc: 0.9221
Epoch 4/20
12600/12600 [==============================] - 8s 649us/sample - loss: 1588.0954 - acc: 0.9241 - val_loss: 1576.2376 - val_acc: 0.9236
Epoch 5/20
12600/12600 [==============================] - 8s 664us/sample - loss: 1414.7711 - acc: 0.9310 - val_loss: 1595.0392 - val_acc: 0.9314
Epoch 6/20
12600/12600 [==============================] - 8s 657us/sample - loss: 1327.7798 - acc: 0.9352 - val_loss: 1240.2152 - val_acc: 0.9371
Epoch 7/20
12600/12600 [==============================] - 8s 657us/sample -

In [0]:
#MODEL TESTING
nets = 4
model = [0] *nets

for j in range(0, 4):
    model[j] = Sequential()
    model[j].add(Conv2D(72,kernel_size=5,activation='relu',input_shape=(image_height, image_width, channels)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(88,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    model[j].add(Dense(2**(j+4), activation='relu'))
    model[j].add(Dense(4, activation='relu'))
    model[j].compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])
    model[j].fit(X_train, y_train, batch_size=64, epochs=25, verbose = 1, validation_data = (X_test, y_test))

Train on 12600 samples, validate on 1400 samples
Epoch 1/25
12600/12600 [==============================] - 6s 505us/sample - loss: 49910.3618 - acc: 0.8948 - val_loss: 43592.1460 - val_acc: 0.9029
Epoch 2/25
12600/12600 [==============================] - 6s 437us/sample - loss: 43301.5581 - acc: 0.9034 - val_loss: 42914.7401 - val_acc: 0.9029
Epoch 3/25
12600/12600 [==============================] - 6s 437us/sample - loss: 42956.0911 - acc: 0.9034 - val_loss: 42741.1792 - val_acc: 0.9029
Epoch 4/25
12600/12600 [==============================] - 6s 438us/sample - loss: 42820.2681 - acc: 0.9034 - val_loss: 42633.2219 - val_acc: 0.9029
Epoch 5/25
12600/12600 [==============================] - 6s 438us/sample - loss: 42736.3291 - acc: 0.9034 - val_loss: 42594.5877 - val_acc: 0.9029
Epoch 6/25
12600/12600 [==============================] - 6s 444us/sample - loss: 42729.0263 - acc: 0.9034 - val_loss: 42626.7588 - val_acc: 0.9029
Epoch 7/25
12600/12600 [==============================] - 6s 45

In [0]:
#MODEL TESTING
nets = 4
model = [0] *nets

for j in range(0, 4):
    model[j] = Sequential()
    model[j].add(Conv2D(72,kernel_size=5,activation='relu',input_shape=(image_height, image_width, channels)))
    model[j].add(MaxPool2D())
    model[j].add(Conv2D(88,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Flatten())
    model[j].add(Dense(2**(j+8), activation='relu'))
    model[j].add(Dense(4, activation='relu'))
    model[j].compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])
    model[j].fit(X_train, y_train, batch_size=64, epochs=25, verbose = 1, validation_data = (X_test, y_test))

Train on 12600 samples, validate on 1400 samples
Epoch 1/25
12600/12600 [==============================] - 7s 543us/sample - loss: 8867.2142 - acc: 0.8787 - val_loss: 2521.0313 - val_acc: 0.9029
Epoch 2/25
12600/12600 [==============================] - 6s 488us/sample - loss: 2164.8041 - acc: 0.9079 - val_loss: 2056.9725 - val_acc: 0.9150
Epoch 3/25
12600/12600 [==============================] - 6s 489us/sample - loss: 1880.0093 - acc: 0.9116 - val_loss: 2138.2900 - val_acc: 0.9157
Epoch 4/25
12600/12600 [==============================] - 6s 489us/sample - loss: 1766.8082 - acc: 0.9164 - val_loss: 1883.7000 - val_acc: 0.9200
Epoch 5/25
12600/12600 [==============================] - 6s 484us/sample - loss: 1608.5838 - acc: 0.9220 - val_loss: 1648.5730 - val_acc: 0.9229
Epoch 6/25
12600/12600 [==============================] - 6s 493us/sample - loss: 1498.8953 - acc: 0.9266 - val_loss: 1390.8563 - val_acc: 0.9300
Epoch 7/25
12600/12600 [==============================] - 6s 476us/sample -

In [0]:
#MODEL TESTING
nets = 3
model = [0] *nets

for j in range(0, 4):
    model[j] = Sequential()
    model[j].add(Conv2D(72,kernel_size=5,activation='relu',input_shape=(image_height, image_width, channels)))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(0.1 + j*0.1))
    model[j].add(Conv2D(88,kernel_size=5,activation='relu'))
    model[j].add(MaxPool2D())
    model[j].add(Dropout(0.1 + j*0.1))
    model[j].add(Flatten())
    model[j].add(Dense(2048, activation='relu'))
    model[j].add(Dropout(0.1 + j*0.1))
    model[j].add(Dense(4, activation='relu'))
    model[j].compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy"])
    model[j].fit(X_train, y_train, batch_size=64, epochs=25, verbose = 1, validation_data = (X_test, y_test))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 12600 samples, validate on 1400 samples
Epoch 1/25
12600/12600 [==============================] - 12s 921us/sample - loss: 8931.9151 - acc: 0.8696 - val_loss: 2979.5025 - val_acc: 0.9029
Epoch 2/25
12600/12600 [==============================] - 11s 852us/sample - loss: 2278.7406 - acc: 0.9075 - val_loss: 2344.2341 - val_acc: 0.9050
Epoch 3/25
12600/12600 [==============================] - 11s 856us/sample - loss: 1972.1963 - acc: 0.9127 - val_loss: 1869.2830 - val_acc: 0.9143
Epoch 4/25
12600/12600 [==============================] - 11s 863us/sample - loss: 1764.6066 - acc: 0.9156 - val_loss: 1561.7902 - val_acc: 0.9179
Epoch 5/25
12600/12600 [==============================] - 11s 859us/sample - loss: 1584.0119 - acc: 0.9209 - val_loss: 1563.6164 - val_acc: 0.9193
Epoch 6/25
12600/12600 [==============================] - 11s 856us/sample - loss: 1446.2138 - acc: 0

IndexError: ignored

In [0]:
#Phase 1 of training
import tensorflow;
checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('model.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
model = Sequential()
model.add(Conv2D(72,kernel_size=5,padding = 'same', activation='relu', input_shape=(image_height, image_width, channels)))
model.add(MaxPool2D())
model.add(Conv2D(88,kernel_size=5,padding = 'same',activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='relu'))
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, batch_size=100, epochs=1000, verbose = 0, callbacks=[checkpoint], validation_data = (X_test, y_test))
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# model.save_weights('model_weights.h5')
weights_file = drive.CreateFile({'title' : 'model_weights1024128.h5'})
weights_file.SetContentFile("model.h5")
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.

Epoch 00001: val_loss improved from inf to 2386.04851, saving model to model.h5

Epoch 00002: val_loss improved from 2386.04851 to 1532.45633, saving model to model.h5

Epoch 00003: val_loss improved from 1532.45633 to 1436.27522, saving model to model.h5

Epoch 00004: val_loss improved from 1436.27522 to 1191.97450, saving model to model.h5

Epoch 00005: val_loss improved from 1191.97450 to 1085.75746, saving model to model.h5

Epoch 00006: val_loss did not improve from 1085.75746

Epoch 00007: val_loss improved from 1085.75746 to 1079.64899, saving model to model.h5

Epoch 00008: val_loss improved from 1079.64899 to 967.61871, saving model to model.h5

Epoch 00009: val_loss improved from 967.61871 to 883.68935, saving model to model.h5

Epoch 00010: val_loss did not improve from 883.68935

Epoch 00011: val_loss did no

GoogleDriveFile({'id': '1EerRjbkSJ93Z2VuJdVwlRnUClVNkGBD3'})

In [9]:
#Phase 2 of training
import tensorflow;
checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('model.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

model.load_weights('model.h5')
model.fit(X_train, y_train, batch_size=100, epochs=1000, verbose = 0, callbacks=[checkpoint], validation_data = (X_test, y_test))

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# model.save_weights('model_weights.h5')
weights_file = drive.CreateFile({'title' : 'final_model_weights.h5'})
weights_file.SetContentFile("model.h5")
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.

Epoch 00001: val_loss improved from inf to 434.43832, saving model to model.h5

Epoch 00002: val_loss improved from 434.43832 to 432.60840, saving model to model.h5

Epoch 00003: val_loss did not improve from 432.60840

Epoch 00004: val_loss did not improve from 432.60840

Epoch 00005: val_loss improved from 432.60840 to 429.39766, saving model to model.h5

Epoch 00006: val_loss did not improve from 429.39766

Epoch 00007: val_loss did not improve from 429.39766

Epoch 00008: val_loss did not improve from 429.39766

Epoch 00009: val_loss did not improve from 429.39766

Epoch 00010: val_loss did not improve from 429.39766

Epoch 00011: val_loss improved from 429.39766 to 429.13579, saving model to model.h5

Epoch 00012: val_loss improved from 429.13579 to 426.33214, saving model to model.h5

Epoch 00013: val_loss did not

GoogleDriveFile({'id': '1ZRcnkQMU2Td9BSEijmD632bwtN28KlLR'})

In [0]:
#Taking input of test file
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#Link :https://drive.google.com/open?id=1omPMyyVytorWoFcV6ab58a2g44S_sRPi
file_id = '1omPMyyVytorWoFcV6ab58a2g44S_sRPi' #link to zip of test images
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_images.zip')
#Link :https://drive.google.com/open?id=1s5CGEidkqP_nw3wpxjmaUDJO-Be382j_
test_id = '1s5CGEidkqP_nw3wpxjmaUDJO-Be382j_' #link to test.csv
myfile = drive.CreateFile({'id': test_id})
myfile.GetContentFile('predict.csv')
test_files = pd.read_csv('predict.csv')
model.load_weights('model.h5')

In [0]:
import zipfile, os
import pandas as pd
import numpy as np
import os, sys
zip_ref = zipfile.ZipFile('test_images.zip', 'r')
zip_ref.extractall('MODEL/')
zip_ref.close()

In [12]:
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
folder = "/content/MODEL/test_images/"
test_files.head()


,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [0]:
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# image_width = 160
# image_height = 120
# channels = 3
dataset = np.ndarray(shape=(len(test_files), image_height, image_width, channels),
                     dtype=np.float32)
i = 0
for file in test_files.loc[:, "image_name"]:
    img = load_img(folder + "/" + file)  # this is a PIL image
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((48, 64, 3))
    x = (x - 128.0) / 128.0
    dataset[i] = x
    i+=1
    

In [0]:
!rm -R /content/MODEL/

In [0]:
!rm -R /content/test_images.zip

In [16]:
result = model.predict(dataset)
result = pd.DataFrame(result)
finalresult = result.round()
test_files.loc[:, "x1"] = finalresult.iloc[:, 0]
test_files.loc[:, "x2"] = finalresult.iloc[:, 1]
test_files.loc[:, "y1"] = finalresult.iloc[:, 2]
test_files.loc[:, "y2"] = finalresult.iloc[:, 3]
test_files.head()

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,236.0,457.0,86.0,408.0
1,1473231475010DeeplearnS11276.png,79.0,565.0,140.0,346.0
2,JPEG_20161205_135307_1000155917326.png,129.0,509.0,42.0,436.0
3,JPEG_20160711_123440_1000518778437.png,211.0,451.0,80.0,424.0
4,JPEG_20160803_115329_100034020722.png,140.0,515.0,41.0,428.0


In [0]:
from google.colab import files
test_files.to_csv("submission.csv", index = False)
files.download("submission.csv")